<a href="https://colab.research.google.com/github/damianmtz98/Datos_Masivos_MCD/blob/main/P6_ModeloPredictivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [13]:
df = pd.read_csv("usdmxn.csv")
df.head(5)

,Date,USD/MXN,USD/EUR,USD/JPY,Oil,Gold,S&P500,DowJones,TasaDesempleoUSA
0,16/07/2018,19.01,1.17,111.43,75.35,1977.5,4565.72,35061.21,3.9
1,23/07/2018,18.61,1.17,111.04,75.42,1960.1,4505.42,34509.03,3.9
2,30/07/2018,18.54,1.16,111.26,73.86,1926.2,4398.95,33734.88,3.9
3,06/08/2018,18.90,1.14,110.87,70.64,1921.1,4450.38,34407.60,3.9
4,13/08/2018,18.88,1.15,110.49,69.16,1919.1,4348.33,33727.43,3.9


In [14]:
df = df.drop(columns=['Date'])
df.head()


,USD/MXN,USD/EUR,USD/JPY,Oil,Gold,S&P500,DowJones,TasaDesempleoUSA
0,19.01,1.17,111.43,75.35,1977.5,4565.72,35061.21,3.9
1,18.61,1.17,111.04,75.42,1960.1,4505.42,34509.03,3.9
2,18.54,1.16,111.26,73.86,1926.2,4398.95,33734.88,3.9
3,18.90,1.14,110.87,70.64,1921.1,4450.38,34407.60,3.9
4,18.88,1.15,110.49,69.16,1919.1,4348.33,33727.43,3.9


In [15]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   USD/MXN           263 non-null    float64
 1   USD/EUR           263 non-null    float64
 2   USD/JPY           263 non-null    float64
 3   Oil               262 non-null    float64
 4   Gold              263 non-null    float64
 5   S&P500            263 non-null    float64
 6   DowJones          263 non-null    float64
 7   TasaDesempleoUSA  263 non-null    float64
dtypes: float64(8)
memory usage: 16.6 KB


(263, 8)

In [16]:
x = df.drop(columns=['USD/MXN'])
y = df['USD/MXN']

In [17]:
#Entrenamiento y prueba
x.entreno, x.prueba, y.entreno, y.prueba = train_test_split(x, y, test_size=0.2, shuffle=True)

<ipython-input-17-93c05c7d481d>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.entreno, x.prueba, y.entreno, y.prueba = train_test_split(x, y, test_size=0.2, shuffle=True)


In [18]:
#Se están eliminando las filas con valores faltantes
x.entreno.dropna(inplace=True)
y.entreno = y.entreno[x.entreno.index]

In [19]:
models = {
    'Gradient Boosting': GradientBoostingRegressor(),

}

results = {}
for name, model in models.items():
    model.fit(x.entreno, y.entreno)
    y.predi = model.predict(x.prueba)
    mse = mean_squared_error(y.prueba, y.predi)
    r2 = r2_score(y.prueba, y.predi)
    results[name] = {'MSE': mse, 'R2 Score': r2}

#Imprimir resultados
print("Resultado:")
for name, result in results.items():
    print(f"{name}:")
    print(f"  MSE: {result['MSE']}")
    print(f"  R2 Score: {result['R2 Score']}")
    print()



Resultado:
Gradient Boosting:
  MSE: 0.18176447889015065
  R2 Score: 0.9029774525600478



In [20]:
predictions = pd.DataFrame({'Real': y.prueba})

# Hacer las predicciones para cada modelo y agregarlas a la tabla
for name, model in models.items():
    model.fit(x.entreno, y.entreno)
    y.predi = model.predict(x.prueba)
    predictions[name] = y.predi

print(predictions)

      Real  Gradient Boosting
249  17.99          17.686418
32   19.25          19.208838
70   19.37          19.133364
229  19.76          19.524745
200  19.85          19.998264
248  17.97          17.695081
186  20.53          20.370431
257  17.15          18.166514
86   21.90          21.194090
237  18.95          19.226588
223  19.79          19.721961
105  22.27          21.829408
96   22.72          23.828835
68   19.12          18.954760
124  19.76          20.170854
152  20.66          19.891062
198  20.13          20.198483
29   19.07          19.299692
175  21.73          20.670948
107  22.38          21.892548
173  20.51          20.603231
57   19.80          19.736174
151  19.86          19.960437
12   18.98          19.164537
88   23.38          22.549074
216  19.88          19.261320
208  20.53          20.310896
141  20.30          20.167101
113  21.13          21.667777
169  20.32          20.330293
49   19.17          19.067576
73   19.00          18.944914
133  20.10

In [21]:
Real=pd.DataFrame(y.prueba)
Real.columns = ["Real"]
Real.head()

,Real
249,17.99
32,19.25
70,19.37
229,19.76
200,19.85


In [23]:
predictions=pd.DataFrame(predictions)
gb = predictions['Gradient Boosting']

predictions_mejores = pd.DataFrame({'Predicción GB': gb})
predictions_mejores.head()

,Predicción GB
249,17.686418
32,19.208838
70,19.133364
229,19.524745
200,19.998264


In [24]:
RealPredictions=pd.concat([predictions_mejores,Real],axis=1)
RealPredictions=pd.DataFrame(RealPredictions)
RealPredictions.head()

,Predicción GB,Real
249,17.686418,17.99
32,19.208838,19.25
70,19.133364,19.37
229,19.524745,19.76
200,19.998264,19.85
